In [1]:
### Libraries
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from keras.models import Model
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
### Loading Data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = np.load(r'top_x_train.npy')
x_test = np.load(r'top_x_test.npy')

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print('Training data shape: ', x_train.shape)
print('Testing data shape : ', x_test.shape)

Training data shape:  (60000, 28, 28)
Testing data shape :  (10000, 28, 28)


In [3]:
input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
### Training AE
from keras.callbacks import TensorBoard
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 21s 358us/step - loss: 0.1190 - val_loss: 0.0825
Epoch 2/50
60000/60000 [==============================] - 17s 288us/step - loss: 0.0710 - val_loss: 0.0671
Epoch 3/50
60000/60000 [==============================] - 17s 289us/step - loss: 0.0619 - val_loss: 0.0593
Epoch 4/50
60000/60000 [==============================] - 17s 289us/step - loss: 0.0575 - val_loss: 0.0584
Epoch 5/50
60000/60000 [==============================] - 17s 288us/step - loss: 0.0551 - val_loss: 0.0546
Epoch 6/50
60000/60000 [==============================] - 17s 289us/step - loss: 0.0534 - val_loss: 0.0518
Epoch 7/50
60000/60000 [==============================] - 17s 287us/step - loss: 0.0521 - val_loss: 0.0496
Epoch 8/50
60000/60000 [==============================] - 17s 287us/step - loss: 0.0510 - val_loss: 0.0501
Epoch 9/50
60000/60000 [======================

In [5]:
### Get Encoded features fron AE
encoder = Model(input_img, encoded)
encoded_imgs_test = encoder.predict(x_test)
encoded_imgs_train = encoder.predict(x_train)
print(encoded_imgs_test.shape)
print(encoded_imgs_train.shape)

(10000, 4, 4, 8)
(60000, 4, 4, 8)


In [6]:
encoded_imgs_test[0]

array([[[0.        , 1.3391603 , 0.80556214, 0.8883792 , 0.60879606,
         0.5196148 , 0.        , 0.87559634],
        [0.        , 1.7058834 , 1.0517826 , 0.5239372 , 0.5595443 ,
         1.4315857 , 0.        , 0.9732729 ],
        [0.        , 1.6621583 , 1.0421363 , 0.6661724 , 0.4315329 ,
         1.1293356 , 0.        , 1.097544  ],
        [0.        , 1.1996729 , 0.12000733, 0.24325125, 0.34706298,
         0.6686797 , 0.        , 0.5562722 ]],

       [[1.1357247 , 0.        , 2.0212932 , 0.        , 1.4265132 ,
         0.77783877, 0.        , 1.0125406 ],
        [2.069103  , 0.        , 2.6168342 , 0.        , 1.3749031 ,
         2.9035566 , 0.        , 1.0050794 ],
        [2.119367  , 0.        , 2.852218  , 0.        , 0.3595068 ,
         3.3854496 , 0.        , 0.89876443],
        [0.88434154, 0.15384819, 0.04768812, 0.        , 0.        ,
         2.2116067 , 0.4100684 , 0.2546988 ]],

       [[0.05639209, 0.1692324 , 0.03239191, 0.2484032 , 0.39363727,
       

In [7]:
encoded_imgs_test = encoded_imgs_test.reshape(10000,128)
encoded_imgs_train = encoded_imgs_train.reshape(60000,128)
print(encoded_imgs_test.shape)
print(encoded_imgs_train.shape)

(10000, 128)
(60000, 128)


In [8]:
np.save('top_test.npy',encoded_imgs_test)
np.save('top_train.npy',encoded_imgs_train)